In [1]:
import os


# main db connection data 
NEO4J_URI = os.environ['NEO4J_URI']
NEO4J_USERNAME = os.environ['NEO4J_USERNAME']
NEO4J_PASSWORD = os.environ['NEO4J_PASSWORD']
NEO4J_DB_NAME = os.environ['NEO4J_DB_NAME']

# ontology db connection data
NEO4J_URI_ONTOLOGY = os.environ['NEO4J_URI_ONTOLOGY']
NEO4J_USERNAME_ONTOLOGY = os.environ['NEO4J_USERNAME_ONTOLOGY']
NEO4J_PASSWORD_ONTOLOGY = os.environ['NEO4J_PASSWORD_ONTOLOGY']
NEO4J_ONTOLOGY_DB_NAME = os.environ['NEO4J_ONTOLOGY_DB_NAME']

In [4]:
from neo4j import GraphDatabase
driver_main = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
driver_ontology = GraphDatabase.driver(NEO4J_URI_ONTOLOGY, auth=(NEO4J_USERNAME_ONTOLOGY, NEO4J_PASSWORD_ONTOLOGY))

In [ ]:
def genPatternDefinedLabelInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (mdl:PatternDefinedLabel)
        RETURN mdl.name AS name, mdl.pattern AS pattern, mdl.classElementVariable AS classElementVariable""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH {record['pattern']} SET {record['classElementVariable']}:{record['name']}"
        yield query

for q in genPatternDefinedLabelInferenceQueries():
    print(q)


In [ ]:
def genPatternDefinedRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (mdr:PatternDefinedRelationship)
        RETURN mdr.name AS name, mdr.pattern AS pattern, mdr.sourceElementVariable AS sourceElementVariable, mdr.targetElementVariable AS targetElementVariable""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH {record['pattern']} MERGE ({record['sourceElementVariable']})-[:{record['name']}]->({record['targetElementVariable']})"
        yield query

for q in genPatternDefinedRelationshipInferenceQueries():
    print(q)


In [ ]:
def genSCOLabelInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (narrower:Label)-[:SCO]->(broader:Label)
        RETURN narrower.name AS narrower, broader.name AS broader""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n:{record['narrower']}) SET n:{record['broader']}"
        yield query

for q in genSCOLabelInferenceQueries():
    print(q)

In [ ]:
def genImpliesRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (narrower:Relationship)-[:IMPLIES]->(broader:Relationship)
        RETURN narrower.name AS narrower, broader.name AS broader""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n)-[:{record['narrower']}]->(m) MERGE (n)-[:{record['broader']}]->(m)"
        yield query

for q in genImpliesRelationshipInferenceQueries():
    print(q)

In [ ]:
def genEquivalentLabelInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (l1:Label)-[:EQUIVALENT]->(l2:Label)
        RETURN l1.name AS l1, l2.name AS l2""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n:{record['l1']}|{record['l2']}) SET n:{record['l1']}:{record['l2']}"
        yield query

for q in genEquivalentLabelInferenceQueries():
    print(q)

In [ ]:
def genEquivalentRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (r1:Relationship)-[:EQUIVALENT]->(r2:Relationship)
        RETURN r1.name AS r1, r2.name AS r2""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n)-[:{record['r1']}|{record['r2']}]->(m) MERGE (n)-[:{record['r1']}]->(m) MERGE (m)-[:{record['r1']}]->(n)"
        yield query

for q in genEquivalentRelationshipInferenceQueries():
    print(q)

In [ ]:
def genSymmetricRelationshipInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (r:Relationship&Symmetric)
        RETURN r.name AS sim_rel""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"MATCH (n)-[:{record['sim_rel']}]->(m) MERGE (m)-[:{record['sim_rel']}]->(n)"
        yield query

for q in genSymmetricRelationshipInferenceQueries():
    print(q)

In [ ]:
def genPatternDefinedNodePropertyInferenceQueries():
    records, summary, keys = driver_ontology.execute_query(
        """MATCH (n:Label)-[:HAS_PROPERTY]->(qdp:PatternDefinedNodeProperty)
        RETURN n.name AS label, qdp.name AS property_name, qdp.pattern AS pattern, qdp.propertyOwnerVariable AS variable, qdp.valueVariable AS val_variable""",
        database_=NEO4J_ONTOLOGY_DB_NAME,
    )
    for record in records:
        query = f"""MATCH ({record['variable']}:{record['label']})
        CALL ({record['variable']}) {{MATCH {record['pattern']}
        WITH {record['variable']}, {record['val_variable']}
        WHERE {record['variable']}.{record['property_name']} IS NULL OR {record['variable']}.{record['property_name']} <> {record['val_variable']}
        SET {record['variable']}.{record['property_name']} = {record['val_variable']} }}
        IN CONCURRENT TRANSACTIONS OF 10 ROWS"""
        yield query

for q in genPatternDefinedNodePropertyInferenceQueries():
    print(q)

In [ ]:
inferenceRulesGenerators = [genPatternDefinedLabelInferenceQueries,
                            genPatternDefinedRelationshipInferenceQueries,
                            genSCOLabelInferenceQueries,
                            genImpliesRelationshipInferenceQueries,
                            genEquivalentLabelInferenceQueries,
                            genEquivalentRelationshipInferenceQueries,
                            genSymmetricRelationshipInferenceQueries,
                            genPatternDefinedNodePropertyInferenceQueries]

def genOntologyInferenceQueries(inferenceRulesGenerators):
    for rule in inferenceRulesGenerators:
        for q in rule():
            yield q

list(genOntologyInferenceQueries(inferenceRulesGenerators))

In [14]:
def infer (rules, params={}):
    """
    This is a function you can use if you want to run a set of inference rules
    until a convergence is reached. why not use it in a RDF-like reasoning context?
    """
    counter = 0
    while True:
        counter += 1
        any_update = False
        for rule in rules:
            with driver_main.session(database=NEO4J_DB_NAME) as session:
                result = session.run(rule, params)
            any_new_update = result.consume().counters._contains_updates
            any_update = any_update or any_new_update
        if not any_update:
            break

In [ ]:
infer(list(genOntologyInferenceQueries(inferenceRulesGenerators)))

In [ ]:
import getopt
import json
import sys
import time
from threading import Thread

from neo4j import GraphDatabase


class CDCService:
    def __init__(self, driver, database, start_cursor=None, selectors=None):
        self.driver = driver
        self.database = database
        self.cursor = start_cursor
        if self.cursor is None:
            self.cursor = self.current_change_id()
        self.selectors = selectors

    def apply_change(self, record):
        record_dict = {
            k: record.get(k)
            for k in ('id', 'txId', 'seq', 'event', 'metadata')
        }
        infer(list(genOntologyInferenceQueries(inferenceRulesGenerators)))

    def query_changes_query(self, tx):
        current = self.current_change_id()
        result = tx.run('CALL db.cdc.query($cursor, $selectors)',
                        cursor=self.cursor, selectors=self.selectors)
        if result.peek() == None:
            self.cursor = current
        else:
            for record in result:
                try:
                    self.apply_change(record)
                except Exception as e:
                    print('Error whilst applying change', e)
                    break
                self.cursor = record['id']

    def query_changes(self):
        with self.driver.session(database=self.database) as session:
            session.execute_read(self.query_changes_query)

    def earliest_change_id(self):
        records, _, _ = self.driver.execute_query(
            'CALL db.cdc.earliest', database_=self.database)
        return records[0]['id']

    def current_change_id(self):
        records, _, _ = self.driver.execute_query(
            'CALL db.cdc.current', database_=self.database)
        return records[0]['id']

    def run(self):
        while True:
            self.query_changes()
            time.sleep(0.5)


def main(argv):
    # Default values
    address = NEO4J_URI
    database = NEO4J_DB_NAME
    username = NEO4J_USERNAME
    password = NEO4J_PASSWORD
    cursor = None

    opts, _ = getopt.getopt(
        argv, 'a:d:u:p:f:',
        ['address=', 'database=', 'username=', 'password=', 'from='])
    for opt, arg in opts:
        if opt in ('-a', '--address'):
            address = arg
        elif opt in ('-d', '--database'):
            database = arg
        elif opt in ('-u', '--username'):
            username = arg
        elif opt in ('-p', '--password'):
            password = arg
        elif opt in ('-f', '--from'):
            cursor = arg

    selectors = [
        # {'select': 'n'}
    ]

    with GraphDatabase.driver(address, auth=(username, password)) as driver:
        cdc = CDCService(driver, database, cursor, selectors)
        cdc_thread = Thread(target=cdc.run, daemon=True)
        cdc_thread.start()
        cdc_thread.join()


#if __name__ == '__main__':
#    main(sys.argv[1:])

main(NEO4J_URI+':'+NEO4J_DB_NAME+':'+NEO4J_USERNAME+':'+NEO4J_PASSWORD+':')